In [2]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import log_loss, mean_squared_error

In [ ]:
from sklearn.metrics import classification_report

# DATA pre-prosessing part

## Load data and labels

#### load training data and lables

In [3]:
train_data = pd.read_csv('orange_small_train.data', sep = '\t')
appetency = pd.read_csv('orange_small_train_appetency_train.labels',header=None).astype('float')
churn = pd.read_csv('orange_small_train_churn_train.labels', sep = '\t').astype('category')
upselling = pd.read_csv('orange_small_train_upselling_train.labels',sep = '\t').astype('category')

In [4]:
df = pd.DataFrame(data = {'-1':[-1]})

In [5]:
appetency

,0
0,-1.0
1,-1.0
2,-1.0
3,-1.0
4,-1.0
5,-1.0
6,-1.0
7,-1.0
8,-1.0
9,-1.0


#### load test data

In [6]:
test_data = pd.read_csv('orange_small_test.data', sep = '\t')
test_appetency = pd.read_csv('orange_small_train_appetency_test.labels',header=None).astype('float')

## Data clean

#### Take a look at the head 10 rows

In [7]:
train_data.head(10)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN
5,NaN,NaN,NaN,NaN,NaN,658.0,7.0,NaN,NaN,NaN,...,zCkv,QqVuch3,LM8l689qOp,NaN,NaN,Qcbd,02N6s8f,Zy3gnGM,am7c,NaN
6,NaN,NaN,NaN,NaN,NaN,1680.0,7.0,NaN,NaN,NaN,...,oslk,XlgxB9z,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN
7,NaN,NaN,NaN,NaN,NaN,77.0,0.0,NaN,NaN,NaN,...,oslk,R2LdzOv,NaN,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,1176.0,7.0,NaN,NaN,NaN,...,zCkv,K2SqEo9,jySVZNlOJy,NaN,kG3k,PM2D,6fzt,am14IcfM7tWLrUmRT52KtA,am7c,NaN
9,NaN,NaN,NaN,NaN,NaN,1141.0,7.0,NaN,NaN,NaN,...,oslk,EPqQcw6,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,NaN,NaN


#### Check if there are too many missing data, draw the loss data picture here TO BE DONE

In [8]:
column_names = train_data.columns

In [9]:
column_names

Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var9',
       'Var10',
       ...
       'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226', 'Var227',
       'Var228', 'Var229', 'Var230'],
      dtype='object', length=230)

In [10]:
####create an 25000 dimentions with 0 array = [0]*250000

In [11]:
###arrayportion_series = pd.Series(0,index = train_data.columns)

In [12]:
###train_data.index train_data.columns

#### Drop the columns whose loss rate > 20%

In [13]:
drop_list = [x for x in column_names if len([ y for y in train_data.loc[:,x].isna() if y == True]) > train_data.shape[0]*0.2]

In [14]:
drop_list

['Var1',
 'Var2',
 'Var3',
 'Var4',
 'Var5',
 'Var8',
 'Var9',
 'Var10',
 'Var11',
 'Var12',
 'Var14',
 'Var15',
 'Var16',
 'Var17',
 'Var18',
 'Var19',
 'Var20',
 'Var23',
 'Var26',
 'Var27',
 'Var29',
 'Var30',
 'Var31',
 'Var32',
 'Var33',
 'Var34',
 'Var36',
 'Var37',
 'Var39',
 'Var40',
 'Var41',
 'Var42',
 'Var43',
 'Var45',
 'Var46',
 'Var47',
 'Var48',
 'Var49',
 'Var50',
 'Var51',
 'Var52',
 'Var53',
 'Var54',
 'Var55',
 'Var56',
 'Var58',
 'Var59',
 'Var60',
 'Var61',
 'Var62',
 'Var63',
 'Var64',
 'Var66',
 'Var67',
 'Var68',
 'Var69',
 'Var70',
 'Var71',
 'Var72',
 'Var75',
 'Var77',
 'Var79',
 'Var80',
 'Var82',
 'Var84',
 'Var86',
 'Var87',
 'Var88',
 'Var89',
 'Var90',
 'Var91',
 'Var92',
 'Var93',
 'Var94',
 'Var95',
 'Var96',
 'Var97',
 'Var98',
 'Var99',
 'Var100',
 'Var101',
 'Var102',
 'Var103',
 'Var104',
 'Var105',
 'Var106',
 'Var107',
 'Var108',
 'Var110',
 'Var111',
 'Var114',
 'Var115',
 'Var116',
 'Var117',
 'Var118',
 'Var120',
 'Var121',
 'Var122',
 'Var124

In [15]:
##train_data.shape??? To judge if data is data frame or series

In [16]:
#####
for x in column_names:
    test_arr = []
    for y in train_data.loc[:,x].isna() :
        if y == False :
            test_arr.append(y)
    #print("lable ="+x)
   # print(len(test_arr))
   # print("\n")

In [17]:
train_data = train_data.drop(drop_list,axis=1)
#test_data = test_data.drop(drop_list,axis=1)

#### Fill the NA of the data type is  int or float data with mean

In [18]:
train_data.mean(axis=0)

Var6      1.330441e+03
Var7      6.780749e+00
Var13     1.259267e+03
Var21     2.346001e+02
Var22     2.902185e+02
Var24     4.566569e+00
Var25     9.676113e+01
Var28     2.247420e+02
Var35     7.477570e-01
Var38     2.566905e+06
Var44     1.642978e-01
Var57     3.508719e+00
Var65     1.476205e+01
Var73     6.629400e+01
Var74     1.039764e+02
Var76     1.488835e+06
Var78     5.364662e-01
Var81     1.025881e+05
Var83     1.985542e+01
Var85     8.494181e+00
Var109    6.140063e+01
Var112    6.602896e+01
Var113   -1.515283e+05
Var119    9.142540e+02
Var123    5.913734e+01
Var125    2.765031e+04
Var132    3.555832e+00
Var133    2.274014e+06
Var134    4.355334e+05
Var140    1.387600e+03
Var143    5.969619e-02
Var144    1.167788e+01
Var149    2.957719e+05
Var153    6.163952e+06
Var160    3.924109e+01
Var163    4.888119e+05
Var173    6.484854e-03
Var181    5.926090e-01
dtype: float64

#### You can see from above that only numberic columns are printed, so just use below function to fill the na numberic values.
#### Tree models do not need

In [19]:
train_data.fillna(train_data.mean())

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
0,1526.0000,7.000000,184.000000,464.000000,580.000000,14.000000,128.000000,166.560000,0.000000,3.570000e+03,...,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,xb3V,RAYp,F2FyR07IdsN7I
1,525.0000,0.000000,0.000000,168.000000,210.000000,2.000000,24.000000,353.520000,0.000000,4.764966e+06,...,NaN,NaN,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,fKCe,RAYp,F2FyR07IdsN7I
2,5236.0000,7.000000,904.000000,1212.000000,1515.000000,26.000000,816.000000,220.080000,0.000000,5.883894e+06,...,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,Qu4f,02N6s8f,ib5G6X1eUxUn6
3,1330.4409,0.000000,0.000000,234.600117,0.000000,4.566569,0.000000,22.080000,0.000000,0.000000e+00,...,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I
4,1029.0000,7.000000,3216.000000,64.000000,80.000000,4.000000,64.000000,200.000000,0.000000,0.000000e+00,...,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I
5,658.0000,7.000000,3156.000000,224.000000,280.000000,2.000000,72.000000,200.000000,5.000000,0.000000e+00,...,ykzL,UYBR,FzaX,KbkKEj0,zCkv,QqVuch3,LM8l689qOp,Qcbd,02N6s8f,Zy3gnGM
6,1680.0000,7.000000,2952.000000,308.000000,385.000000,4.000000,128.000000,176.560000,0.000000,1.315800e+04,...,NvHl,UYBR,FzaX,JO03372,oslk,XlgxB9z,LM8l689qOp,FSa2,RAYp,55YFVY9
7,77.0000,0.000000,0.000000,32.000000,40.000000,2.000000,16.000000,230.560000,0.000000,3.776496e+06,...,9haV,UYBR,NaN,U8IKsQe,oslk,R2LdzOv,NaN,FSa2,RAYp,F2FyR07IdsN7I
8,1176.0000,7.000000,2912.000000,200.000000,250.000000,2.000000,64.000000,300.320000,0.000000,6.014460e+06,...,NaN,NaN,FzaX,ROeipLp,zCkv,K2SqEo9,jySVZNlOJy,PM2D,6fzt,am14IcfM7tWLrUmRT52KtA
9,1141.0000,7.000000,164.000000,208.000000,260.000000,2.000000,72.000000,166.560000,5.000000,5.317974e+06,...,b30L,UYBR,FzaX,fabLnWA,oslk,EPqQcw6,LM8l689qOp,FSa2,RAYp,55YFVY9


In [20]:
train_data.dtypes.describe()

count          66
unique          3
top       float64
freq           37
dtype: object

#### Convert the string columns to category type. Need an analysis figure to illustrate why category can be chosen.

In [21]:
object_columns = train_data.select_dtypes(['object']).columns 

In [22]:
train_data[object_columns] = train_data[object_columns].apply(lambda col: col.astype('category'))
#test_data[object_columns] = test_data[object_columns].apply(lambda col: col.astype('category'))

In [23]:
train_data.dtypes

Var6       float64
Var7       float64
Var13      float64
Var21      float64
Var22      float64
Var24      float64
Var25      float64
Var28      float64
Var35      float64
Var38      float64
Var44      float64
Var57      float64
Var65      float64
Var73        int64
Var74      float64
Var76      float64
Var78      float64
Var81      float64
Var83      float64
Var85      float64
Var109     float64
Var112     float64
Var113     float64
Var119     float64
Var123     float64
Var125     float64
Var132     float64
Var133     float64
Var134     float64
Var140     float64
            ...   
Var173     float64
Var181     float64
Var192    category
Var193    category
Var195    category
Var196    category
Var197    category
Var198    category
Var199    category
Var202    category
Var203    category
Var204    category
Var205    category
Var206    category
Var207    category
Var208    category
Var210    category
Var211    category
Var212    category
Var216    category
Var217    category
Var218    ca

In [24]:
train_data['Var223'].cat.categories

Index(['LM8l689qOp', 'M_8D', 'bCPvVye', 'jySVZNlOJy'], dtype='object')

In [25]:
len(train_data.select_dtypes(['category']).columns)

28

#### Convert the category to codes. Not needed for tree models

In [26]:
train_data[object_columns] = train_data[object_columns].apply(lambda x: x.cat.codes)
#test_data[object_columns] = test_data[object_columns].apply(lambda x: x.cat.codes)

#### Convert the category type to float. Not needed for tree models

In [27]:
#train_data[object_columns] = train_data[object_columns].apply(lambda x: x.astype('int'))
#test_data[object_columns] = test_data[object_columns].apply(lambda x: x.astype('int'))

#### Fill the NA with the most frequent item TO BE DONE for non-tree classifier.
#### Tree models do not need.
#### Do one hot encoding. TO BE DONE 

# LGB part

In [28]:
#### lgb load training and testing data

In [29]:
#lgb_train_data = lgb.Dataset(data=train_data, label=appetency)
#lgb_test_data = lgb.Dataset(data=test_data,label=test_appetency,reference=lgb_train_data)

In [30]:
####lgb parameters

In [31]:
param = {'num_leaves': 31, 'objective': 'binary','metric':'auc'}

In [32]:
####Training lgb model

In [33]:
for ty in train_data.dtypes:
    print(ty)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int16
int8
int8
int8
int16
int16
int16
int16
int8
int8
int8
int8
int8
int8
int8
int8
int8
int16
int16
int8
int8
int16
int8
int16
int8
int8
int8
int8


In [34]:
#num_round = 10
#bst = lgb.train(param, lgb_train_data, num_round, valid_sets=[lgb_test_data])#,categorical_feature=object_columns, valid_sets=[lgb_test_data]

In [35]:
#pred = bst.predict(test_data)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(train_data, appetency, test_size=0.1, random_state=42)

In [37]:
 gbm = lgb.LGBMClassifier(n_estimators=50, silent=True)
 gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=5, verbose=False)
 result = gbm.predict_proba(X_test)
 print(result)

C:\Users\mocui\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mocui\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[0.99239161 0.00760839]
 [0.99278104 0.00721896]
 [0.99278104 0.00721896]
 ...
 [0.98359583 0.01640417]
 [0.9627041  0.0372959 ]
 [0.97786718 0.02213282]]


In [38]:
len(result)

2500

In [39]:
result2 = gbm.predict(X_test)

In [40]:
print(result2)

[-1. -1. -1. ... -1. -1. -1.]


In [41]:
len([x for x in result2 if x == -1])

2500

In [53]:
y_train.describe()

,0
count,22500.000000
mean,-0.964533
std,0.263967
min,-1.000000
25%,-1.000000
50%,-1.000000
75%,-1.000000
max,1.000000


In [54]:
from sklearn.metrics import classification_report

In [59]:
print(classification_report(y_test,result2))

              precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99      2461
         1.0       0.00      0.00      0.00        39

   micro avg       0.98      0.98      0.98      2500
   macro avg       0.49      0.50      0.50      2500
weighted avg       0.97      0.98      0.98      2500



In [50]:
len([x for x in y_train if x != -1.0])

1

In [ ]:
 ret = log_loss(y_test, gbm.predict_proba(X_test))
 print(ret)
